リカレントニューラルネットワーク（RNN） のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。


フォワードプロパゲーションの実装を必須課題とし、バックプロパゲーションの実装はアドバンス課題とします。


クラスの名前はScratchSimpleRNNClassifierとしてください。クラスの構造などは以前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にしてください。



## 【問題1】SimpleRNNのフォワードプロパゲーション実装

SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。


フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。


バッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。


$$a_t = x_{t}\cdot W_{x} + h_{t-1}\cdot W_{h} + B\\
h_t = tanh(a_t)$$


$
a_t
$ : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)

$
h_t
$ : 時刻tの状態・出力 (batch_size, n_nodes)


$x_{t}
$ : 時刻tの入力 (batch_size, n_features)


$W_{x}
 $: 入力に対する重み (n_features, n_nodes)


$h_{t-1}
 $: 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)


$W_{h}
 $: 状態に対する重み。 (n_nodes, n_nodes)


$B
 $: バイアス項 (n_nodes,)



 初期状態 $
h_{0}
 $は全て0とすることが多いですが、任意の値を与えることも可能です。


上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力 $
x$
 は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。


分類問題であれば、それぞれの時刻のhに対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。タスクによっては最後の時刻のhだけを使用することもあります。

In [51]:
import numpy as np

In [52]:
class ScratchSimpleRNNClassifier():

    def __init__(self):
        pass

    def forward(self, x, w_x, w_h):
        self.batch_size = x.shape[0] # 1
        self.n_sequences = x.shape[1] # 3
        self.n_features = x.shape[2] # 2
        self.n_nodes = w_x.shape[1] # 4
        h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
        b = np.array([1, 1, 1, 1]) # (n_nodes,)

        for i in range(n_sequences):
            if i ==0:
                a = np.dot(x[0,i,:], w_x) + np.dot(h, w_h) + b
                a = np.tanh(a)
            else:
                a = np.dot(x[0,i,:], w_x) + np.dot(a, w_h) + b
                a = np.tanh(a)
                
        return a

## 【問題2】小さな配列でのフォワードプロパゲーションの実験
小さな配列でフォワードプロパゲーションを考えてみます。


入力x、初期状態h、重みw_xとw_h、バイアスbを次のようにします。


ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。


```
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)
```

フォワードプロパゲーションの出力が次のようになることを作成したコードで確認してください。


```
h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) # (batch_size, n_nodes)
```




In [53]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)

In [54]:
display(x.shape)
display(w_x.shape)
display(w_h.shape)
display(h.shape)

(1, 3, 2)

(2, 4)

(4, 4)

(1, 4)

In [55]:
testa = np.dot(x[0,0,:], w_x) + np.dot(h, w_h) + b
testa = np.tanh(testa)
print(testa)

testb = np.dot(x[0,1,:], w_x) + np.dot(testa, w_h) + b
testb = np.tanh(testb)
print(testb)

testc = np.dot(x[0,2,:], w_x) + np.dot(testb, w_h) + b
testc = np.tanh(testc)
print(testc)

[[0.76188798 0.76213958 0.76239095 0.76255841]]
[[0.792209   0.8141834  0.83404912 0.84977719]]
[[0.79494228 0.81839002 0.83939649 0.85584174]]


In [56]:
for i in range(n_sequences):
    if i ==0:
        a = np.dot(x[0,i,:], w_x) + np.dot(h, w_h) + b
        a = np.tanh(a)
        print("i==0 :{}".format(a))
    else:
        a = np.dot(x[0,i,:], w_x) + np.dot(a, w_h) + b
        a = np.tanh(a)
        print("i !=0 : {}".format(a))

i==0 :[[0.76188798 0.76213958 0.76239095 0.76255841]]
i !=0 : [[0.792209   0.8141834  0.83404912 0.84977719]]
i !=0 : [[0.79494228 0.81839002 0.83939649 0.85584174]]


In [57]:
ssr = ScratchSimpleRNNClassifier()
h = ssr.forward(x, w_x, w_h)
h

array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

### 答え

```
h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) # (batch_size, n_nodes)
```

## 【問題3】（アドバンス課題）バックプロパゲーションの実装

バックプロパゲーションを実装してください。


RNNの内部は全結合層を組み合わせた形になっているので、更新式は全結合層などと同様です。

$$W_x^{\prime} = W_x - \alpha \frac{\partial L}{\partial W_x} \\
W_h^{\prime} = W_h - \alpha \frac{\partial L}{\partial W_h} \\
B^{\prime} = B - \alpha \frac{\partial L}{\partial B}$$

$α$
 : 学習率

 $\frac{\partial L}{\partial W_x}$
: $
W
x
 $に関する損失 $
L
$ の勾配

 $\frac{\partial L}{\partial W_h}$
 : 
$W_{h}$
 に関する損失 $
L
$ の勾配

$\frac{\partial L}{\partial B}$
: $
B
 $に関する損失 $
L
$ の勾配

勾配を求めるためのバックプロパゲーションの数式が以下です。

$\frac{\partial h_t}{\partial a_t} = \frac{\partial L}{\partial h_t} ×(1-tanh^2(a_t))$

$\frac{\partial L}{\partial B} = \frac{\partial h_t}{\partial a_t}$

$\frac{\partial L}{\partial W_x} = x_{t}^{T}\cdot \frac{\partial h_t}{\partial a_t}$

$\frac{\partial L}{\partial W_h} = h_{t-1}^{T}\cdot \frac{\partial h_t}{\partial a_t}$

$\frac{\partial L}{\partial h_t}$は前の時刻からの状態の誤差と出力の誤差の合計です。hは順伝播時に出力と次の層に伝わる状態双方に使われているからです。


前の時刻や層に流す誤差の数式は以下です。

$\frac{\partial L}{\partial h_{t-1}} = \frac{\partial h_t}{\partial a_t}\cdot W_{h}^{T}$

$\frac{\partial L}{\partial x_{t}} = \frac{\partial h_t}{\partial a_t}\cdot W_{x}^{T}$